$ \frac{1}{4} \sum_{m<n} J_{HS}(m-n) (\sigma^x_{m} \sigma^x_{n} + \sigma^y_{m} \sigma^y_{n} + \sigma^z_{m} \sigma^z_n) $ where $J_{HS}(m-n) = \frac{\\pi^2}{N^2 \sin^2 ((m-n)\pi / N)}$, $N$ is number of sites. Use periodic boundary conditions

In [2]:
import numpy as np
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmay, sigmaz
import matplotlib.pyplot as plt

In [3]:
def wrap_index(N, m, n):
    if m >= n:
        swap = m
        m = n
        n = swap
    
    forward_dist = abs(m-n)
    back_dist = abs(N+m-n)
    return min(forward_dist, back_dist)

In [10]:
nqs_en_x = []
nqs_en_y = []
analytical_en_x = []
analytical_en_y = []
for N in range(3, 51):
    if N %2 == 0:
        hi = nk.hilbert.Spin(s=1/2, N=N)
        def haldane_shastry_model(N):
            sum = 0
            for n in range(N):
                for m in range(n):
                    hs = (np.pi**2) / (N**2 * (np.sin((wrap_index(N, n, m))*np.pi/N)**2))
                    x = sigmax(hi, m) * sigmax(hi, n)
                    y = sigmay(hi, m) * sigmay(hi, n)
                    z = sigmaz(hi, m) * sigmaz(hi, n)
                    sum += hs * (x+y+z)
            return sum / 4
        hs20 = haldane_shastry_model(N)
        rbm = nk.models.RBM(alpha=1, param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.01))
        sampler = nk.sampler.MetropolisLocal(hi)
        vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)
        optimizer = nk.optimizer.Sgd(learning_rate=0.1)
        preconditioner = nk.optimizer.SR(diag_shift=0.1, holomorphic=False)
        vmc = nk.driver.VMC(hs20, optimizer, variational_state=vstate, preconditioner=preconditioner)
        log = nk.logging.RuntimeLog()
        vmc.run(n_iter=300, out=log)
        opt_energy = vstate.expect(hs20)
        nqs_en_x.append(N)
        analytical_en_x.append(N)
        nqs_en_y.append(opt_energy.mean)
        analytical_en_y.append(-np.pi**2/24*(N+5/N))
plt.plot(nqs_en_x, nqs_en_y, label="NQS") 
plt.plot(analytical_en_x, analytical_en_y, label="analytical") 
plt.legend()
    

/Users/lzawbrito/opt/miniconda3/envs/nqs/lib/python3.12/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples
100%|██████████| 300/300 [00:01<00:00, 203.59it/s, Energy=-2.15863642-0.00002563j ± 0.00000091 [σ²=0.00000000, R̂=1.4031]]
/Users/lzawbrito/opt/miniconda3/envs/nqs/lib/python3.12/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples
 92%|█████████▏| 277/300 [00:02<00:00, 109.11it/s, Energy=-2.8196-0.0181j ± 0.0018 [σ²=0.0017, R̂=1.4031]]    


KeyboardInterrupt: 

In [98]:
hs20 = haldane_shastry_model(N)

In [22]:
rbm = nk.models.RBM(alpha=3, param_dtype=complex, kernel_init=nn.initializers.normal(stddev=0.001))
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)
optimizer = nk.optimizer.Sgd(learning_rate=0.05)
preconditioner = nk.optimizer.SR(diag_shift=0.1)
vmc = nk.driver.VMC(hs20, optimizer, variational_state=vstate, preconditioner=preconditioner)
log = nk.logging.RuntimeLog()
vmc.run(n_iter=300, out=log)
opt_energy = vstate.expect(hs20)
print(opt_energy)

/Users/lzawbrito/opt/miniconda3/envs/nqs/lib/python3.12/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples
  0%|          | 0/300 [00:00<?, ?it/s]/Users/lzawbrito/opt/miniconda3/envs/nqs/lib/python3.12/site-packages/netket/optimizer/qgt/qgt_onthefly.py:107: HolomorphicUndeclaredWarning: 
Defaulting to `holomorphic=False`, but this might lead to increased
computational cost or disabled features. Check if your variational
function is holomorphic, and if so specify `holomorphic=True`as an extra
keyword argument.

To silence this warning, specify the `holomorphic=False/True` keyword
argument.

To numerically check whether your variational function is or not holomorphic
you can use the following snippet:

```python
   vs = nk.vqs.MCState(...)

   nk.utils.is_probably_holomorphic(vs._apply_fun, vs.parameters, vs.samples, vs.model_state)
```

-7.2436-0.0039j ± 0.0095 [σ²=0.0457, R̂=1.0777]


In [21]:
hs_energy = lambda x: -np.pi**2 * (x + 5/x)/24 
true_energy = range(3,N+1)
true_energy = [hs_energy(x) for x in true_energy]
print(true_energy)
hs_energy(N)

[-1.9190897446562643, -2.158975962738297, -2.4674011002723395, -2.8100956975323865, -3.1723728432072935, -3.546889081641488, -3.9295647152485405, -4.317951925476594, -4.71049300961083, -5.106149499174703, -5.50420245445368, -5.9041383470802415, -6.305580589584868, -6.708246741365422, -7.111920818432036, -7.516434833237034, -7.921656164032249, -8.327478713419145]


-8.327478713419145